# BTModel

> This is a simple implementation of a latent binomial variable model for binomial outcome data.

In [ ]:
#| default_exp BTModel

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pymc as pm
import xarray as xr
import numpy as np

In [ ]:
#| export
class BTModel:
    """
    Model implements a pymc model for Bayesian brand tracking
    
    Initialization parameters:
    
    positive_outcome_name: str = Name of the positive outcome variable
        
    sample_size_name: str
    """
    
    def __init__(self, 
                 positive_outcome_name: str,
                 sample_size_name: str,
                 exogenous_variables: list[str]):
      
        self.positive_outcome_name = positive_outcome_name
        self.sample_size_name = sample_size_name
        self.exogenous_variables = exogenous_variables
        self.__model = None

    def fit(self, data: xr.Dataset):
        """
        Fit the model to the data
        
        Parameters:
        
        data: xr.Dataset = Dataset with the data
        """
        # Data Coords
        errors = []
        if self.positive_outcome_name not in data:
            errors.append(ValueError(f"Positive outcome variable {self.positive_outcome_name} not found in data"))
        if self.sample_size_name not in data:
            errors.append(ValueError(f"Sample size variable {self.sample_size_name} not found in data"))
        for var in self.exogenous_variables:
            if var not in data:
                errors.append(ValueError(f"Exogenous variable {var} not found in data"))
        if len(errors) > 0:
            raise ValueError(errors)
        
        coords = {"exogenous_variables": self.exogenous_variables}
        data_dims = data.dims
        data_coords = {dim: data[dim].values for dim in data.dims}
        coords.update(data_coords)
        
        
        # Define the model
        with pm.Model(coords=coords) as self.__model:
            # Define the priors
            beta = pm.Normal('beta', mu=0, sd=1, shape=len(self.exogenous_variables))
            alpha = pm.Normal('alpha', mu=0, sd=1)
            sigma = pm.HalfNormal('sigma', sd=1)
            
            # Define the likelihood
            mu = alpha + pm.math.dot(data[self.exogenous_variables].values, beta)
            p = pm.math.invlogit(mu)
            y = pm.Binomial('y', n=data[self.sample_size_name].values, p=p, observed=data[self.positive_outcome_name].values)
            
            # Sample the model
            self.__trace = pm.sample(5000, tune=1000)

    def predict(self, X):
        pass

    def evaluate(self, X, y):
        pass

    def save(self, path):
        pass

    def load(self, path):
        pass

In [ ]:
#| hide

xr.Dataset({"positive_outcome": (["time"], np.random.randint(0, 100, 100)),})

<xarray.Dataset> Size: 800B
Dimensions:           (time: 100)
Dimensions without coordinates: time
Data variables:
    positive_outcome  (time) int64 800B 15 81 13 78 79 64 ... 82 85 39 78 67 84

In [ ]:
bt_model = BTModel("positive_outcome", "sample_size", ["exog1", "exog2"])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()